In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/salvicnlp-2025/Test Data/BG/input-file.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240904_EUZ_BG_46.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240627_EUZ_BG_64.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240417_URW_BG_79.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240301_SHG_BG_59.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240726_EUZ_BG_51.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240216_URW_BG_92.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20231124_IHC_BG_35.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240216_URW_BG_89.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240321_URW_BG_82.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240327_EUZ_BG_98.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240321_URW_BG_66.txt
/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents/20240327_EUZ_BG_96

## Train Data

In [87]:
import pandas as pd
import os

# Paths
ANNOTATION_PATH = '/kaggle/input/salvicnlp-2025/Train Data/BG/subtask-1-annotations.txt'
RAW_TEXT_DIR = '/kaggle/input/salvicnlp-2025/Train Data/BG/raw-documents'

# Read annotation file
df = pd.read_csv(
    ANNOTATION_PATH,
    sep='\t',
    header=None,
    names=['file_name', 'start', 'end', 'label']
)

# Prepare a column for the extracted text
df['text'] = ''

for file_name in df['file_name'].unique():
    file_path = os.path.join(RAW_TEXT_DIR, file_name)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    with open(file_path, 'r', encoding='utf-8') as f:
        full_text = f.read()
    for idx, row in df[df['file_name'] == file_name].iterrows():
        start = int(row['start'])
        end = int(row['end'])
        # Find the previous and next newline
        prev_nl = full_text.rfind('\n', 0, start)
        next_nl = full_text.find('\n', end)
        # Adjust indices to include the full line
        span_start = prev_nl + 1 if prev_nl != -1 else 0
        span_end = next_nl if next_nl != -1 else len(full_text)
        sentence = full_text[span_start:span_end].strip()
        df.at[idx, 'text'] = sentence

print(df[['file_name', 'start', 'end', 'label', 'text']].head())

                file_name  start   end  label  \
0  20240704_URW_BG_30.txt      0    61  False   
1  20240704_URW_BG_30.txt     65   141  False   
2  20240704_URW_BG_30.txt    145   208  False   
3  20240704_URW_BG_30.txt    212   645  False   
4  20240704_URW_BG_30.txt    649  1110  False   

                                                text  
0  ПРЕДСТАВЯНЕ НА ПОЗИЦИЯТА НА РЕПУБЛИКА БЪЛГАРИЯ...  
1  Моля, от вносителите, заповядайте, госпожо Вас...  
2  ВИКТОРИЯ ВАСИЛЕВА (ВЕЛИЧИЕ): Благодаря Ви, гос...  
3  Уважаеми господин служебен Министър-председате...  
4  Не трябва да забравяме, че ангажираността и чл...  


In [88]:
# Function to keep only text before first double newline
def remove_after_double_newline(text):
    if pd.isna(text):
        return text
    return text.split('\n\n', 1)[0]

# Apply the function to the 'text' column
df['text'] = df['text'].apply(remove_after_double_newline)

In [89]:
df.head()

,file_name,start,end,label,text
0,20240704_URW_BG_30.txt,0,61,False,ПРЕДСТАВЯНЕ НА ПОЗИЦИЯТА НА РЕПУБЛИКА БЪЛГАРИЯ...
1,20240704_URW_BG_30.txt,65,141,False,"Моля, от вносителите, заповядайте, госпожо Вас..."
2,20240704_URW_BG_30.txt,145,208,False,"ВИКТОРИЯ ВАСИЛЕВА (ВЕЛИЧИЕ): Благодаря Ви, гос..."
3,20240704_URW_BG_30.txt,212,645,False,Уважаеми господин служебен Министър-председате...
4,20240704_URW_BG_30.txt,649,1110,False,"Не трябва да забравяме, че ангажираността и чл..."


In [90]:
len(df)

363

In [91]:
df.iloc[300]['text']

'Ей, хора, с тази тема не си играйте, ще взривите държавата! Каквото има, трябва да го знаят всички. Не може само Борисов, Пеевски и Петков да знаят, а другите да стоим и да гадаем какво се случва. Всички имаме достъп до информация и всички ще четем – не лидери, не председатели на комисии, всички народни представители. Темата е сериозна и не се подигравайте с българските граждани.'

## Trail Data

In [92]:
import pandas as pd
import os

# Paths
ANNOTATION_PATH = '/kaggle/input/salvicnlp-2025/TRIAL-ANNOTATED/BG/subtask-1-annotations.txt'
RAW_TEXT_DIR = '/kaggle/input/salvicnlp-2025/TRIAL-ANNOTATED/BG/raw-documents'

# Read annotation file
trial_df = pd.read_csv(
    ANNOTATION_PATH,
    sep='\t',
    header=None,
    names=['file_name', 'start', 'end', 'label']
)

# Prepare a column for the extracted text
trial_df['text'] = ''

for file_name in trial_df['file_name'].unique():
    file_path = os.path.join(RAW_TEXT_DIR, file_name)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    with open(file_path, 'r', encoding='utf-8') as f:
        full_text = f.read()
    for idx, row in trial_df[trial_df['file_name'] == file_name].iterrows():
        start = int(row['start'])
        end = int(row['end'])
        # Find the previous and next newline
        prev_nl = full_text.rfind('\n', 0, start)
        next_nl = full_text.find('\n', end)
        # Adjust indices to include the full line
        span_start = prev_nl + 1 if prev_nl != -1 else 0
        span_end = next_nl if next_nl != -1 else len(full_text)
        sentence = full_text[span_start:span_end].strip()
        trial_df.at[idx, 'text'] = sentence

print(trial_df[['file_name', 'start', 'end', 'label', 'text']].head())

        file_name  start   end  label  \
0  trail_BG_1.txt      0    57  False   
1  trail_BG_1.txt     61   413  False   
2  trail_BG_1.txt    417   996  False   
3  trail_BG_1.txt   1000  1581  False   
4  trail_BG_1.txt   1585  2168  False   

                                                text  
0  АНДРЕЙ ЧОРБАНОВ (ИТН): Благодаря Ви, госпожо П...  
1  Уважаеми дами и господа народни представители!...  
2  Ще обясня накратко защо това е така. Какво пре...  
3  А какво ще бъде то? Ако имаме десет цени, на к...  
4  Втори обаче много по-важен въпрос е липсата на...  


In [93]:
# Function to keep only text before first double newline
def remove_after_double_newline(text):
    if pd.isna(text):
        return text
    return text.split('\n\n', 1)[0]

# Apply the function to the 'text' column
trial_df['text'] = trial_df['text'].apply(remove_after_double_newline)

In [94]:
trial_df.head()

,file_name,start,end,label,text
0,trail_BG_1.txt,0,57,False,"АНДРЕЙ ЧОРБАНОВ (ИТН): Благодаря Ви, госпожо П..."
1,trail_BG_1.txt,61,413,False,Уважаеми дами и господа народни представители!...
2,trail_BG_1.txt,417,996,False,Ще обясня накратко защо това е така. Какво пре...
3,trail_BG_1.txt,1000,1581,False,"А какво ще бъде то? Ако имаме десет цени, на к..."
4,trail_BG_1.txt,1585,2168,False,Втори обаче много по-важен въпрос е липсата на...


In [95]:
len(trial_df)

75

In [96]:
df = pd.concat([df, trial_df], ignore_index=True)

In [97]:
df.head()

,file_name,start,end,label,text
0,20240704_URW_BG_30.txt,0,61,False,ПРЕДСТАВЯНЕ НА ПОЗИЦИЯТА НА РЕПУБЛИКА БЪЛГАРИЯ...
1,20240704_URW_BG_30.txt,65,141,False,"Моля, от вносителите, заповядайте, госпожо Вас..."
2,20240704_URW_BG_30.txt,145,208,False,"ВИКТОРИЯ ВАСИЛЕВА (ВЕЛИЧИЕ): Благодаря Ви, гос..."
3,20240704_URW_BG_30.txt,212,645,False,Уважаеми господин служебен Министър-председате...
4,20240704_URW_BG_30.txt,649,1110,False,"Не трябва да забравяме, че ангажираността и чл..."


In [98]:
len(df)

438

## Test Data

In [99]:
import pandas as pd
import os

# Paths
ANNOTATION_PATH = '/kaggle/input/salvicnlp-2025/Test Data/BG/input-file.txt'
RAW_TEXT_DIR = '/kaggle/input/salvicnlp-2025/Test Data/BG/raw-documents'

# Read annotation file
test_df = pd.read_csv(
    ANNOTATION_PATH,
    sep='\t',
    header=None,
    names=['file_name', 'start', 'end']
)

# Prepare a column for the extracted text
test_df['text'] = ''

for file_name in test_df['file_name'].unique():
    file_path = os.path.join(RAW_TEXT_DIR, file_name)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    with open(file_path, 'r', encoding='utf-8') as f:
        full_text = f.read()
    for idx, row in test_df[test_df['file_name'] == file_name].iterrows():
        start = int(row['start'])
        end = int(row['end'])
        # Find the previous and next newline
        prev_nl = full_text.rfind('\n', 0, start)
        next_nl = full_text.find('\n', end)
        # Adjust indices to include the full line
        span_start = prev_nl + 1 if prev_nl != -1 else 0
        span_end = next_nl if next_nl != -1 else len(full_text)
        sentence = full_text[span_start:span_end].strip()
        test_df.at[idx, 'text'] = sentence

print(test_df[['file_name', 'start', 'end', 'text']].head())

                file_name  start   end  \
0  20240228_URW_BG_86.txt      0   190   
1  20240228_URW_BG_86.txt    192   574   
2  20240228_URW_BG_86.txt    576  1258   
3  20240228_URW_BG_86.txt   1260  1503   
4  20240228_URW_BG_86.txt   1505  1560   

                                                text  
0  ЯВОР БОЖАНКОВ (ПП-ДБ): Уважаеми господин Предс...  
1  Докато тя извършва агресивни действия в Украйн...  
2  Депутат от ГЕРБ зададе въпрос към министър Таг...  
3  Вместо да сложим край на това днес и да кажем,...  
4  КОСТАДИН КОСТАДИНОВ (ВЪЗРАЖДАНЕ, от място): Не...  


In [100]:
# Apply the function to the 'text' column
test_df['text'] = test_df['text'].apply(remove_after_double_newline)

In [101]:
len(test_df)

1361

In [102]:
%%capture
%pip install Dataset
%pip install transformers
%pip install sentencepiece
%pip install datasets
%pip install huggingface_hub
%pip install bitsandbytes
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [103]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer, 
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset
import os
from tqdm import tqdm

In [104]:
# Set seed for reproducibility
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HUGGINGFACE_TOKEN = user_secrets.get_secret("HUGGINGFACE_TOKEN")

!huggingface-cli login --token $HUGGINGFACE_TOKEN

In [ ]:
len(df)

In [ ]:
train_df = df[:350]
eval_df = df[351:]

In [ ]:
print(f"Training set size: {len(train_df)}")
print(f"Evaluation set size: {len(eval_df)}")
print(f"Test set size: {len(test_df)}")

## Check Token Legth

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('FacebookAI/xlm-roberta-large')

In [ ]:
# Compute token lengths for each text
token_lengths = train_df['text'].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=True)))
max_token_length = token_lengths.max()
avg_token_length = token_lengths.mean()
print(max_token_length)
print(avg_token_length)

In [ ]:
# Compute token lengths for each text
test_df['token_length'] = test_df['text'].apply(lambda x: len(tokenizer.encode(x, add_special_tokens=True)))

# Plot histogram of token lengths
plt.figure(figsize=(10,6))
plt.hist(test_df['token_length'], bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Token Lengths in train_df["text"]')
plt.xlabel('Token Length')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
train_df['label'] = train_df['label'].map({True: 1, False: 0})
eval_df['label'] = eval_df['label'].map({True: 1, False: 0})

In [ ]:
print(f"Number of labels in train {train_df['label'].value_counts()}")
print(f"Number of labels in eval {eval_df['label'].value_counts()}")

In [ ]:
# Function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
train_dataset = train_df.reset_index(drop=True)
eval_dataset = eval_df.reset_index(drop=True)

## Models

In [ ]:
import optuna
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset  # Hugging Face Datasets
import numpy as np

In [ ]:
# 1. Load a small dataset for demonstration (replace with your own)
dataset = load_dataset("imdb", split="train[:2000]")
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [ ]:
# 2. Tokenization
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
def tokenize(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=128)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True)
eval_dataset = eval_dataset.map(tokenize, batched=True)

In [ ]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
# 3. Metric function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    accuracy = (preds == labels).mean()
    return {"accuracy": accuracy}

In [ ]:
# 4. Optuna objective function
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)
    batch_size = trial.suggest_categorical("batch_size", [8, 16])

    training_args = TrainingArguments(
        output_dir="./results",
        eval_strategy="epoch",
        save_strategy="no",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=1,  # Use 1 for quick search; increase for real runs
        weight_decay=weight_decay,
        logging_steps=10,
        disable_tqdm=True,
        report_to="none"
    )

    model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_accuracy"]

In [ ]:
# 5. Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)  # Increase n_trials for a more thorough search